Data source:
Kaggle. (2012). Titanic: Machine Learning from Disaster. Retrieved from https://www.kaggle.com/c/titanic/data.

In this tutorial, we will walk through the process of getting a decision tree-based explanation for model predictions on the Titanic dataset. We will see which features contribute most to survival predictions for passengers who were on board the Titanic. The data loading and pre-processing steps are identical to tutorials of the global and local feature contributions.


In [ ]:
import pandas as pd
import numpy as np

np.random.seed(123)

We will begin by loading the data, and writing out human-readable descriptions of all features.

In [ ]:
from sklearn.model_selection import train_test_split
from pyreal.applications import titanic

np.random.seed(1)

x_orig, y = titanic.load_titanic_data()

x_train_orig, x_test_orig, y_train, y_test = train_test_split(x_orig, y, test_size=.20)

print("Features:", x_orig.columns.values)

feature_descriptions = {
    "PassengerId": "Passenger ID",
    "Pclass": "Ticket Class",
    "SibSp": "Number of siblings/spouses aboard",
    "Parch": "Number of parents/children aboard",
    "Ticket": "Ticket Number",
    "Fare": "Passenger Fare",
    "Cabin": "Cabin Number",
    "Embarked": "Port of Embarkment",
}

In [ ]:
from pyreal.transformers import OneHotEncoder, ColumnDropTransformer, MultiTypeImputer
from pyreal.transformers import fit_transformers, run_transformers

column_drop = ColumnDropTransformer(["PassengerId", "Name", "Ticket", "Cabin"])
imputer = MultiTypeImputer()
one_hot_encoder = OneHotEncoder(["Sex", "Embarked"])

transformers = [column_drop, imputer, one_hot_encoder]
fit_transformers(transformers, x_train_orig)
x_transform_train = run_transformers(transformers, x_train_orig)
x_transform_test = run_transformers(transformers, x_test_orig)

Next, we will make the transformers. `Pyreal`'s built-in transformers all have explanation transforms, which bring explanations from the model form to the interpretable form.

In [ ]:
from sklearn.linear_model import LogisticRegression
import pickle

model = LogisticRegression(max_iter=500)
model.fit(x_transform_train, y_train)

preds = model.predict(x_transform_test)
print("Test accuracy: %.2f" % (np.mean(preds==y_test)*100))

Now, we can make the decision tree explainer...

In [ ]:
from pyreal.explainers import DecisionTreeExplainer

dte = DecisionTreeExplainer(model=model, x_train_orig=x_train_orig, is_classifier=True, max_depth=5,
                            m_transformers=transformers, e_transformers=transformers,
                            feature_descriptions=feature_descriptions, fit_on_init=True)

The `produce()` function of a `DecisionTreeExplainer` class returns a `sklearn.tree.DecisionTreeClassifier` or `DecisionTreeRegressor` object. We can also visualize the explanation by plotting the explanation tree.

To better make sense of the tree diagram, we can call `dte.return_features()` to help the visualization.

In [ ]:
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt

decision_tree = dte.produce()

feature_names = dte.return_features()

fig, ax = plt.subplots(figsize=(30,30))

plot_tree(decision_tree, feature_names=feature_names, class_names=["died", "survived"],
              impurity=False, fontsize=10, ax=ax, filled=True)
plt.show()

The decision tree can also tell us how each feature contributes to the prediction of the survival of the passengers.

In [ ]:
from pyreal.utils import visualize

importances = dte.produce_importances()

visualize.plot_top_contributors(importances, select_by="absolute")